# Database Design and Load Exercise

### Steps
 1. Analyze - Completed last module
 2. Design - Completed last module. 
 3. Data Carpentry
 4. Data Loading
 5. Analytical Queries

# Smart Stores ERD

In Module 2 module everyone developed an ERD diagram for a smart store based on the entities and attributes below.



### orders :
* `order_id`: order identifier
* `user_id`: customer identifier
* `eval_set`: which evaluation set this order belongs in (see `SET` described below)
* `order_number`: the order sequence number for this user (1 = first, n = nth)
* `order_dow`: the day of the week the order was placed on
* `order_hour_of_day`: the hour of the day the order was placed on
* `days_since_prior`: days since the last order, capped at 30 (with NAs for `order_number` = 1)

### products :
* `product_id`: product identifier
* `product_name`: name of the product
* `aisle_id`: foreign key
* `department_id`: foreign key

### aisles :
* `aisle_id`: aisle identifier
* `aisle`: the name of the aisle

### deptartments :
* `department_id`: department identifier
* `department`: the name of the department

### order_products :
* `order_id`: foreign key
* `product_id`: foreign key
* `add_to_cart_order`: order in which each product was added to cart
* `reordered`: 1 if this product has been ordered by this user in the past, 0 otherwise

#### where SET is one of the four following evaluation sets (eval_set in orders):

* "prior": orders prior to that users most recent order (~3.2m orders)
* "train": training data supplied to participants (~131k orders)
* "test": test data reserved for machine learning competitions (~75k orders)

# ERD Diagram

For this assignment this is the ERD diagram everyone should use based on the requirements above. It is ok if you had something a little bit different in last week's exercise as long as you captured all of the requirements and relationships.

![erd](../images/smart_store.png)


### 1. Create the tables in the database using SQLAlchemy

Convert the Entities and attributes into a Database schema for Postgres. **Remember to specify your Primary Keys and Foreign Keys for each table!** 

In [42]:
import getpass
mypasswd = getpass.getpass()
username = 'rc25g'
host = 'pgsql.dsa.lan'
database = 'dsa_student'

········


In [43]:
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine

# SQLAlchemy Connection Parameters
postgres_db = {'drivername': 'postgres',
               'username': username,
               'password': mypasswd,
               'host': host,
               'database' :database}
engine = create_engine(URL(**postgres_db), echo=False)
del mypasswd

In [78]:
query = """
DROP TABLE IF EXISTS orders;
CREATE TABLE orders (
    order_id               INT NOT NULL, 
    user_id                INT NOT NULL, 
    eval_set               varchar(100), 
    order_number           INT NOT NULL,
    order_dow              INT NOT NULL,
    order_hour_of_day      INT NOT NULL,
    days_since_prior       INT NOT NULL,
     PRIMARY KEY (order_id)
)
"""

with engine.connect() as connection:
    res = connection.execute(query)
    print(res)

2020-12-06 14:31:13,545 INFO sqlalchemy.engine.base.Engine 
DROP TABLE IF EXISTS orders;
CREATE TABLE orders (
    order_id               INT NOT NULL, 
    user_id                INT NOT NULL, 
    eval_set               varchar(100), 
    order_number           INT NOT NULL,
    order_dow              INT NOT NULL,
    order_hour_of_day      INT NOT NULL,
    days_since_prior       INT NOT NULL,
     PRIMARY KEY (order_id)
)

2020-12-06 14:31:13,547 INFO sqlalchemy.engine.base.Engine {}
2020-12-06 14:31:13,558 INFO sqlalchemy.engine.base.Engine COMMIT


In [80]:
query = """
DROP TABLE IF EXISTS order_products;
CREATE TABLE order_products (
    order_id                 INT NOT NULL,
    product_id               INT NOT NULL, 
    add_to_cart_order        INT NOT NULL, 
    reordered                INT NOT NULL, 
    CONSTRAINT FK_1 FOREIGN KEY (order_id)
         REFERENCES orders(order_id), 
    CONSTRAINT FK_2 FOREIGN KEY (product_id)
         REFERENCES products(product_id), 
     PRIMARY KEY (order_id, product_id)
)
"""

with engine.connect() as connection:
    res = connection.execute(query)
    print(res)

2020-12-06 14:31:32,836 INFO sqlalchemy.engine.base.Engine 
DROP TABLE IF EXISTS order_products;
CREATE TABLE order_products (
    order_id                 INT NOT NULL,
    product_id               INT NOT NULL, 
    add_to_cart_order        INT NOT NULL, 
    reordered                INT NOT NULL, 
    CONSTRAINT FK_1 FOREIGN KEY (order_id)
         REFERENCES orders(order_id), 
    CONSTRAINT FK_2 FOREIGN KEY (product_id)
         REFERENCES products(product_id), 
     PRIMARY KEY (order_id, product_id)
)

2020-12-06 14:31:32,837 INFO sqlalchemy.engine.base.Engine {}
2020-12-06 14:31:32,848 INFO sqlalchemy.engine.base.Engine COMMIT


In [73]:
query = """
DROP TABLE IF EXISTS products;
CREATE TABLE products (
    product_id               INT NOT NULL, 
    department_id            INT NOT NULL, 
    aisle_id                 INT NOT NULL, 
    product_name             varchar(100),
    CONSTRAINT FK_1 FOREIGN KEY (department_id)
         REFERENCES departments(department_id), 
    CONSTRAINT FK_2 FOREIGN KEY (aisle_id)
         REFERENCES aisles(aisle_id), 
     PRIMARY KEY (product_id)
)
"""

with engine.connect() as connection:
    res = connection.execute(query)
    print(res)

2020-12-06 14:24:22,621 INFO sqlalchemy.engine.base.Engine 
DROP TABLE IF EXISTS products;
CREATE TABLE products (
    product_id               INT NOT NULL, 
    department_id            INT NOT NULL, 
    aisle_id                 INT NOT NULL, 
    product_name             varchar(100),
    CONSTRAINT FK_1 FOREIGN KEY (department_id)
         REFERENCES departments(department_id), 
    CONSTRAINT FK_2 FOREIGN KEY (aisle_id)
         REFERENCES aisles(aisle_id), 
     PRIMARY KEY (product_id)
)

2020-12-06 14:24:22,623 INFO sqlalchemy.engine.base.Engine {}
2020-12-06 14:24:22,632 INFO sqlalchemy.engine.base.Engine COMMIT


In [70]:
query = """
DROP TABLE IF EXISTS aisles;
CREATE TABLE aisles (
    aisle_id               INT NOT NULL,
    aisle                  varchar(100), 
     PRIMARY KEY (aisle_id)
)
"""

with engine.connect() as connection:
    res = connection.execute(query)
    print(res)

2020-12-06 14:24:07,432 INFO sqlalchemy.engine.base.Engine 
DROP TABLE IF EXISTS aisles;
CREATE TABLE aisles (
    aisle_id               INT NOT NULL,
    aisle                  varchar(100), 
     PRIMARY KEY (aisle_id)
)

2020-12-06 14:24:07,433 INFO sqlalchemy.engine.base.Engine {}
2020-12-06 14:24:07,486 INFO sqlalchemy.engine.base.Engine COMMIT


In [59]:
query = """
DROP TABLE IF EXISTS departments;
CREATE TABLE departments (
    department_id               INT NOT NULL,
    department                  varchar(100), 
     PRIMARY KEY (department_id)
)
"""

with engine.connect() as connection:
    res = connection.execute(query)
    print(res)

2020-12-06 14:17:22,751 INFO sqlalchemy.engine.base.Engine 
DROP TABLE IF EXISTS departments;
CREATE TABLE departments (
    department_id               INT NOT NULL,
    department                  varchar(100), 
     PRIMARY KEY (department_id)
)

2020-12-06 14:17:22,752 INFO sqlalchemy.engine.base.Engine {}
2020-12-06 14:17:22,761 INFO sqlalchemy.engine.base.Engine COMMIT


In [37]:
import pandas as pd

### 2. Load data from the following files using SQLAlchemy:

#### `/dsa/data/all_datasets/instacart/orders.csv`
 * 3421084 Rows
 * File Preview 
```
order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
2539329,1,prior,1,2,08,
2398795,1,prior,2,3,07,15.0
473747,1,prior,3,3,12,21.0
2254736,1,prior,4,4,07,29.0
```

#### `/dsa/data/all_datasets/instacart/products.csv`
 * 49689 Rows
 * File Preview 
```
product_id,product_name,aisle_id,department_id
1,Chocolate Sandwich Cookies,61,19
2,All-Seasons Salt,104,13
3,Robust Golden Unsweetened Oolong Tea,94,7
4,Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce,38,1
```

#### `/dsa/data/all_datasets/instacart/aisles.csv`
 * 135 Rows
 * File Preview 
```
aisle_id,aisle
1,prepared soups salads
2,specialty cheeses
3,energy granola bars
4,instant foods
```

#### `/dsa/data/all_datasets/instacart/departments.csv`
 * 22 Rows
 * File Preview 
```
department_id,department
1,frozen
2,other
3,bakery
4,produce
```

#### `/dsa/data/all_datasets/instacart/order_products.csv`
 * 1384618 Rows
 * File Preview 
```
order_id,product_id,add_to_cart_order,reordered
1,49302,1,1
1,11109,2,1
1,10246,3,0
1,49683,4,0
```
     

## In each designated cell, load the data using Python



### 2.1 - Orders 

In [38]:
orders = pd.read_csv('/dsa/data/all_datasets/instacart/orders.csv')

orders['days_since_prior_order'] = orders['days_since_prior_order'].fillna(0)

In [ ]:
orders.to_sql('orders',
          engine,
          schema= username,
          if_exists='append'
          index=False,
          chunksize=50)

### 2.2 - Products







In [74]:
products = pd.read_csv('/dsa/data/all_datasets/instacart/products.csv')


In [77]:
products.to_sql('products',
          engine,
          schema= username,
          if_exists='append',
          index=False,
          chunksize=50)

2020-12-06 14:24:52,024 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2020-12-06 14:24:52,025 INFO sqlalchemy.engine.base.Engine {'schema': 'rc25g', 'name': 'products'}
2020-12-06 14:24:52,031 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-06 14:24:52,033 INFO sqlalchemy.engine.base.Engine INSERT INTO rc25g.products (product_id, product_name, aisle_id, department_id) VALUES (%(product_id)s, %(product_name)s, %(aisle_id)s, %(department_id)s)
2020-12-06 14:24:52,034 INFO sqlalchemy.engine.base.Engine ({'product_id': 1, 'product_name': 'Chocolate Sandwich Cookies', 'aisle_id': 61, 'department_id': 19}, {'product_id': 2, 'product_name': 'All-Seasons Salt', 'aisle_id': 104, 'department_id': 13}, {'product_id': 3, 'product_name': 'Robust Golden Unsweetened Oolong Tea', 'aisle_id': 94, 'department_id': 7}, {'product_id': 4, 'product_name': 'Smart Ones Classic Favorites 

2020-12-06 14:24:52,123 INFO sqlalchemy.engine.base.Engine INSERT INTO rc25g.products (product_id, product_name, aisle_id, department_id) VALUES (%(product_id)s, %(product_name)s, %(aisle_id)s, %(department_id)s)
2020-12-06 14:24:52,124 INFO sqlalchemy.engine.base.Engine ({'product_id': 301, 'product_name': 'Biotin 10,000mcg Maximum Strength', 'aisle_id': 47, 'department_id': 11}, {'product_id': 302, 'product_name': 'Gourmet Dog Treats Bone', 'aisle_id': 40, 'department_id': 8}, {'product_id': 303, 'product_name': 'Candy, Original', 'aisle_id': 45, 'department_id': 19}, {'product_id': 304, 'product_name': 'Dry Roasted Pistachios', 'aisle_id': 117, 'department_id': 19}, {'product_id': 305, 'product_name': 'Wild Forest Raw Neem Honey', 'aisle_id': 29, 'department_id': 13}, {'product_id': 306, 'product_name': 'Gold Potatoes', 'aisle_id': 83, 'department_id': 4}, {'product_id': 307, 'product_name': 'Precious Mozzarella Cheese', 'aisle_id': 21, 'department_id': 16}, {'product_id': 308, 'pro

2020-12-06 14:24:52,187 INFO sqlalchemy.engine.base.Engine INSERT INTO rc25g.products (product_id, product_name, aisle_id, department_id) VALUES (%(product_id)s, %(product_name)s, %(aisle_id)s, %(department_id)s)
2020-12-06 14:24:52,188 INFO sqlalchemy.engine.base.Engine ({'product_id': 601, 'product_name': 'Organic Crunchy Slaw Americana Salad', 'aisle_id': 123, 'department_id': 4}, {'product_id': 602, 'product_name': 'Banana Chocolate Chip', 'aisle_id': 52, 'department_id': 1}, {'product_id': 603, 'product_name': 'NasaFlo Neti Pot Sinus Relief with Premixed', 'aisle_id': 11, 'department_id': 11}, {'product_id': 604, 'product_name': 'Cookie Sheets', 'aisle_id': 10, 'department_id': 17}, {'product_id': 605, 'product_name': 'Natural Liquid Laundry Detergent - Blue Eucalyptus and Lavender', 'aisle_id': 75, 'department_id': 17}, {'product_id': 606, 'product_name': 'Salted Caramel Ice Cream', 'aisle_id': 37, 'department_id': 1}, {'product_id': 607, 'product_name': 'Goober Strawberry', 'ais

2020-12-06 14:24:52,257 INFO sqlalchemy.engine.base.Engine INSERT INTO rc25g.products (product_id, product_name, aisle_id, department_id) VALUES (%(product_id)s, %(product_name)s, %(aisle_id)s, %(department_id)s)
2020-12-06 14:24:52,258 INFO sqlalchemy.engine.base.Engine ({'product_id': 901, 'product_name': 'Dark Chocolate Peanut Candy', 'aisle_id': 45, 'department_id': 19}, {'product_id': 902, 'product_name': 'Baked Rice Crisps, Sour Cream & Wasabi, Medium', 'aisle_id': 78, 'department_id': 19}, {'product_id': 903, 'product_name': 'Gluten Free Sugar Cones', 'aisle_id': 103, 'department_id': 19}, {'product_id': 904, 'product_name': 'Tofu, Gluten Free, Garlic Pepper', 'aisle_id': 14, 'department_id': 20}, {'product_id': 905, 'product_name': 'Fish Oil with Vitamin D 1000mg', 'aisle_id': 47, 'department_id': 11}, {'product_id': 906, 'product_name': 'Chamomilla 30 C', 'aisle_id': 47, 'department_id': 11}, {'product_id': 907, 'product_name': 'Premium Sliced Bacon', 'aisle_id': 106, 'departm

2020-12-06 14:24:52,331 INFO sqlalchemy.engine.base.Engine INSERT INTO rc25g.products (product_id, product_name, aisle_id, department_id) VALUES (%(product_id)s, %(product_name)s, %(aisle_id)s, %(department_id)s)
2020-12-06 14:24:52,331 INFO sqlalchemy.engine.base.Engine ({'product_id': 1201, 'product_name': 'Maxi Super with Flexi-Wings', 'aisle_id': 126, 'department_id': 11}, {'product_id': 1202, 'product_name': 'Organic Whole Grain Oatmeal Cereal Baby Food', 'aisle_id': 92, 'department_id': 18}, {'product_id': 1203, 'product_name': 'Crunchy Organic Dark Roasted Peanut Butter', 'aisle_id': 88, 'department_id': 13}, {'product_id': 1204, 'product_name': 'Rain Bath Refreshing Shower Gel', 'aisle_id': 127, 'department_id': 11}, {'product_id': 1205, 'product_name': 'Gourmet Nacho Cheddar Cheese Sauce', 'aisle_id': 30, 'department_id': 6}, {'product_id': 1206, 'product_name': 'Noodle Soup, Shin Cup, Gourmet Spicy', 'aisle_id': 66, 'department_id': 6}, {'product_id': 1207, 'product_name': 'G

2020-12-06 14:24:52,404 INFO sqlalchemy.engine.base.Engine INSERT INTO rc25g.products (product_id, product_name, aisle_id, department_id) VALUES (%(product_id)s, %(product_name)s, %(aisle_id)s, %(department_id)s)
2020-12-06 14:24:52,404 INFO sqlalchemy.engine.base.Engine ({'product_id': 1501, 'product_name': 'Mexican Blend', 'aisle_id': 104, 'department_id': 13}, {'product_id': 1502, 'product_name': 'Acai Berry Lip Balm', 'aisle_id': 132, 'department_id': 11}, {'product_id': 1503, 'product_name': 'Low Fat Cottage Cheese', 'aisle_id': 108, 'department_id': 16}, {'product_id': 1504, 'product_name': 'Goat Milk Cheese', 'aisle_id': 21, 'department_id': 16}, {'product_id': 1505, 'product_name': 'Zero Calorie Fruit Punch Sports Drink', 'aisle_id': 64, 'department_id': 7}, {'product_id': 1506, 'product_name': 'Sliced Baby Bellas Mushrooms', 'aisle_id': 123, 'department_id': 4}, {'product_id': 1507, 'product_name': 'Positively Radiant Brightening Cleanser', 'aisle_id': 73, 'department_id': 11}

2020-12-06 14:24:52,470 INFO sqlalchemy.engine.base.Engine INSERT INTO rc25g.products (product_id, product_name, aisle_id, department_id) VALUES (%(product_id)s, %(product_name)s, %(aisle_id)s, %(department_id)s)
2020-12-06 14:24:52,471 INFO sqlalchemy.engine.base.Engine ({'product_id': 1801, 'product_name': 'Pomegranate Blueberry Pistachio Plus Antioxidants 1.4 oz Fruit & Nut Bars', 'aisle_id': 3, 'department_id': 19}, {'product_id': 1802, 'product_name': 'Vitamin C-500 mg Tablets', 'aisle_id': 47, 'department_id': 11}, {'product_id': 1803, 'product_name': 'Original Bagel Crisps Roasted Garlic', 'aisle_id': 107, 'department_id': 19}, {'product_id': 1804, 'product_name': 'Baked White Cheddar Cheese Puffs', 'aisle_id': 107, 'department_id': 19}, {'product_id': 1805, 'product_name': 'Organic Lemon Agave Ginger Super Juice', 'aisle_id': 98, 'department_id': 7}, {'product_id': 1806, 'product_name': 'Strawberry Unsweetened Soft Drink Mix', 'aisle_id': 90, 'department_id': 7}, {'product_id':

2020-12-06 14:24:52,534 INFO sqlalchemy.engine.base.Engine INSERT INTO rc25g.products (product_id, product_name, aisle_id, department_id) VALUES (%(product_id)s, %(product_name)s, %(aisle_id)s, %(department_id)s)
2020-12-06 14:24:52,535 INFO sqlalchemy.engine.base.Engine ({'product_id': 2101, 'product_name': 'Chicken & Mushroom Aussie Pies', 'aisle_id': 38, 'department_id': 1}, {'product_id': 2102, 'product_name': 'Hash Browns Country Style', 'aisle_id': 52, 'department_id': 1}, {'product_id': 2103, 'product_name': 'ProteinPLUS Multigrain Penne Pasta', 'aisle_id': 131, 'department_id': 9}, {'product_id': 2104, 'product_name': 'Healthy Kitten Formula Dry Cat Food', 'aisle_id': 41, 'department_id': 8}, {'product_id': 2105, 'product_name': 'Salted Caramel Chocolate Cookie Dough', 'aisle_id': 105, 'department_id': 13}, {'product_id': 2106, 'product_name': 'Earl Grey Decaffeinated Black Tea', 'aisle_id': 94, 'department_id': 7}, {'product_id': 2107, 'product_name': 'Caramel Apple Filled Har

2020-12-06 14:24:52,610 INFO sqlalchemy.engine.base.Engine INSERT INTO rc25g.products (product_id, product_name, aisle_id, department_id) VALUES (%(product_id)s, %(product_name)s, %(aisle_id)s, %(department_id)s)
2020-12-06 14:24:52,611 INFO sqlalchemy.engine.base.Engine ({'product_id': 2401, 'product_name': 'Organic Milk Chocolate Baking Wafers', 'aisle_id': 17, 'department_id': 13}, {'product_id': 2402, 'product_name': 'Beef Lo Mein', 'aisle_id': 38, 'department_id': 1}, {'product_id': 2403, 'product_name': 'Almond Butter, Vanilla Espresso', 'aisle_id': 88, 'department_id': 13}, {'product_id': 2404, 'product_name': 'Tamari Roasted Pumpkin Seeds', 'aisle_id': 6, 'department_id': 2}, {'product_id': 2405, 'product_name': 'Organic Just Pears Sage 1 Baby Food', 'aisle_id': 92, 'department_id': 18}, {'product_id': 2406, 'product_name': 'Pure Zero', 'aisle_id': 64, 'department_id': 7}, {'product_id': 2407, 'product_name': 'Mild Smoked Snack Sticks', 'aisle_id': 23, 'department_id': 19}, {'p

DataError: (psycopg2.errors.StringDataRightTruncation) value too long for type character varying(100)

[SQL: INSERT INTO rc25g.products (product_id, product_name, aisle_id, department_id) VALUES (%(product_id)s, %(product_name)s, %(aisle_id)s, %(department_id)s)]
[parameters: ({'product_id': 2551, 'product_name': 'Butterscotch Pudding Snacks', 'aisle_id': 71, 'department_id': 16}, {'product_id': 2552, 'product_name': 'Organic Raw Pomegranate Kombucha', 'aisle_id': 94, 'department_id': 7}, {'product_id': 2553, 'product_name': 'Rosemary Flatbread Kamut', 'aisle_id': 78, 'department_id': 19}, {'product_id': 2554, 'product_name': 'Snacking Lupini Beans Hot Pepper', 'aisle_id': 100, 'department_id': 21}, {'product_id': 2555, 'product_name': 'Bread Bohemian Style Rye Unseeded', 'aisle_id': 112, 'department_id': 3}, {'product_id': 2556, 'product_name': 'Yellow Mustard', 'aisle_id': 72, 'department_id': 13}, {'product_id': 2557, 'product_name': 'Touch Of Phlly Shredded Sharp Cheddar Cheese', '

### 2.3 - Aisles

In [71]:
aisles = pd.read_csv('/dsa/data/all_datasets/instacart/aisles.csv')


In [72]:
aisles.to_sql('aisles',
          engine,
          schema= username,
          if_exists='append',
          index=False,
          chunksize=50)

2020-12-06 14:24:16,034 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2020-12-06 14:24:16,035 INFO sqlalchemy.engine.base.Engine {'schema': 'rc25g', 'name': 'aisles'}
2020-12-06 14:24:16,039 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-06 14:24:16,040 INFO sqlalchemy.engine.base.Engine INSERT INTO rc25g.aisles (aisle_id, aisle) VALUES (%(aisle_id)s, %(aisle)s)
2020-12-06 14:24:16,041 INFO sqlalchemy.engine.base.Engine ({'aisle_id': 1, 'aisle': 'prepared soups salads'}, {'aisle_id': 2, 'aisle': 'specialty cheeses'}, {'aisle_id': 3, 'aisle': 'energy granola bars'}, {'aisle_id': 4, 'aisle': 'instant foods'}, {'aisle_id': 5, 'aisle': 'marinades meat preparation'}, {'aisle_id': 6, 'aisle': 'other'}, {'aisle_id': 7, 'aisle': 'packaged meat'}, {'aisle_id': 8, 'aisle': 'bakery desserts'}  ... displaying 10 of 50 total bound parameter sets ...  {'aisle_id': 49, 'aisle':

### 2.4 - Departments

In [60]:
departments = pd.read_csv('/dsa/data/all_datasets/instacart/departments.csv')


In [66]:
departments.to_sql('departments',
          engine,
          schema= username,
          if_exists='append',
          index=False,
          chunksize=50)

2020-12-06 14:20:29,543 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2020-12-06 14:20:29,544 INFO sqlalchemy.engine.base.Engine {'schema': 'rc25g', 'name': 'departments'}
2020-12-06 14:20:29,548 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-06 14:20:29,549 INFO sqlalchemy.engine.base.Engine INSERT INTO rc25g.departments (department_id, department) VALUES (%(department_id)s, %(department)s)
2020-12-06 14:20:29,549 INFO sqlalchemy.engine.base.Engine ({'department_id': 1, 'department': 'frozen'}, {'department_id': 2, 'department': 'other'}, {'department_id': 3, 'department': 'bakery'}, {'department_id': 4, 'department': 'produce'}, {'department_id': 5, 'department': 'alcohol'}, {'department_id': 6, 'department': 'international'}, {'department_id': 7, 'department': 'beverages'}, {'department_id': 8, 'department': 'pets'}  ... displaying 10 of 21 total bound paramet

### 2.5 - Order Products

In [15]:
order_products = pd.read_csv('/dsa/data/all_datasets/instacart/order_products.csv')


In [ ]:
order_products.to_sql('order_products',
          engine,
          schema= username,
          if_exists='append'
          index=False,
          chunksize=50)

# Save your notebook, then `File > Close and Halt`

---